# **Deep Hedging**
# Buchkov Viacheslav

In [6]:
from pathlib import Path

from deep_hedging import ExperimentConfig, EuropeanCall, seed_everything
from deep_hedging.dl import SpotDatasetTexts, TrainerTexts, Assessor
from deep_hedging.dl.models import LSTMHedgerTexts
from deep_hedging.dl.baselines import BaselineEuropeanCall

RANDOM_SEED = 12

In [2]:
config = ExperimentConfig(
    RANDOM_SEED=RANDOM_SEED,
    DATA_ROOT=Path("data"),
    DATA_FILENAME="data_with_texts",
    OUTPUT_ROOT=Path("."),
)
seed_everything(RANDOM_SEED)

config.DEVICE

device(type='mps')

In [3]:
config

ExperimentConfig(N_DAYS=5, REBAL_FREQ='30 min', VAR_QUANTILE=0.05, TEST_SIZE=0.1, RANDOM_SEED=12, DATA_ROOT=PosixPath('data'), OUTPUT_ROOT=PosixPath('.'), DATA_FILENAME='data_with_texts', LAYER_NORM=False, USE_TIME_DIFF=True, USE_SPOT_START=True, N_EPOCHS=20, N_STEPS_RL_TRAIN=1000000, LR=0.01, BATCH_SIZE=32, NUM_LAYERS=3, HIDDEN_DIM=32, EMBED_MAX_DIM=128, OPTIMIZER=<class 'torch.optim.adam.Adam'>, NUM_WORKERS=2)

## Dataset.

In [ ]:
spot_dataset = SpotDatasetTexts(
    n_days=config.N_DAYS, instrument_cls=EuropeanCall, config=config
)
spot_dataset.df.head()

In [ ]:
trainer = TrainerTexts(
    model_cls=LSTMHedgerTexts, instrument_cls=EuropeanCall, config=config
)
trainer.run(1)

In [ ]:
assessor = Assessor(
    model=trainer.hedger,
    baseline=BaselineEuropeanCall(dt=trainer.dt).to(config.DEVICE),
    test_loader=trainer.test_loader,
)
assessor.run()

In [ ]:
trainer.save(config.OUTPUT_ROOT)